<img src="images/strathsdr_banner.png" align="left">

# Hardware Accelerated Spectrum Analysis on RFSoC
----

<div class="alert alert-box alert-info">
Please use Jupyter Labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents a flexible hardware accelerated Spectrum Analyzer Module for the Zynq UltraScale+ RFSoC. The Spectrum Analyzer Module was developed by the [University of Strathclyde](https://github.com/strath-sdr).

## Table of Contents
* [Introduction](#introduction)
    * [Hardware Setup](#hardware-setup)
    * [Software Setup](#software-setup)
* [Simple Tone Generation](#simple-tone-generation)
* [The Spectrum Analyzer](#the-spectrum-analyzer)
    * [A Simple Example](#a-simple-example)
* [Conclusion](#conclusion)
    
## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.4, November 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf)

## Revision History
* **v1.0** | 12/02/2021 | Spectrum analyzer notebook
* **v1.1** | 15/04/2021 | Update spectral resolution and minimum bandwidth with new value
* **v1.2** | 19/05/2023 | Update for ZCU208

----

## Introduction <a class="anchor" id="introduction"></a>
The Zynq RFSoC contains high frequency samplers known as RF Data Converters (RF DCs). The RF DCs are tightly coupled with the Programmable Logic (PL), creating a high-throughput, low-latency path between the FPGA and analogue world. The Spectrum Analyzer Module employs the RF Analogue-to-Digital Converters (RF ADCs) to receive RF time domain signals. The received data is manipulated using spectral pre-processing techniques in the PL, to prepare it for frequency domain analysis and visualisation in the Processing System (PS).

A significant portion of the design has been implemented in the RFSoC's PL to prevent the PS from applying highly computational arithemtic. There is a Spectrum Analyzer Module for each available RF ADC channel in the design. The Spectrum Analyzers are also interfaced to their very own flexible decimator, allowing different sample rates to be configured for each channel.

### Hardware Setup <a class="anchor" id="hardware-setup"></a>
The ZCU208 development boards require the SMA and clock connections to be setup correctly. We will begin with connecting the clocks from the Clk104 card to the development board. Then connect the ADCs and DACs to the HW_XM_655 card's RF baluns.

Before you begin with the setup below, ensure that the Clk104 card and HW_XM_655 card is securely connected to the ZCU208 development board.

#### CLK104 Setup <a class="anchor" id="clk104_setup"></a>
On the Clk104 board, make the following connections as shown in [Figure 1](#fig-1).

<a class="anchor" id="fig-4"></a>
<figure>
<img src='images/clk104_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 1: Setup of the clk104 add-on board on the ZCU208.</b></figcaption>
</figure>

Ensure that the clocks above are connected as follows:
* DAC_229_CLK to DAC_RFCLK_B (P & N).
* ADC_225_CLK to ADC_RFCLK_B (P & N).

If you do not make the correct connection, the demonstration will not operate correctly.

#### ADC Setup <a class="anchor" id="adc_setup"></a>
The next step is to connect the ADCs to the RF Baluns on the HW_XM_655 card. See the image below in [Figure 2](#fig-2) on how to make these connections successfully. You will need the rectangle connector that contains a strip of wires with SMAs attached to make this connection. This connector is known as the Carlisle SMA 8 Cable Assemblies in the kit contents page supplied with your development board.

<a class="anchor" id="fig-2"></a>
<figure>
<img src='images/zcu208_adc_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 2: Setting-up the ADC on the HW_XM_655 card.</b></figcaption>
</figure>

Ensure that the ADC connections are setup as follows:
* JHC5 P0_224 to J61
* JHC5 N01_224 to J63
* JHC5 P2_224 to J2
* JHC5 N23_224 to J6
* JHC5 P0_225 to J1
* JHC5 N01_225 to J5
* JHC5 P2_225 to J18
* JHC5 N23_225 to J16

This step is a little difficult as there are many small wires. Please take your time setting up the ADCs to prevent errors.

#### DAC Setup <a class="anchor" id="dac_setup"></a>
The DACs are connected to the RF Baluns on the HW_XM_655 card using the connections shown in [Figure 3](#fig-3) below.

<a class="anchor" id="fig-3"></a>
<figure>
<img src='images/zcu208_dac_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 3: Setting-up the DAC on the HW_XM_655 card.</b></figcaption>
</figure>

Ensure that the DAC connections are setup as follows:
* JHC3 P0_230 to J64
* JHC3 N0_230 to J66
* JHC3 P2_230 to J28
* JHC3 N2_230 to J30
* JHC3 P0_231 to J26
* JHC3 N0_231 to J29
* JHC3 P2_231 to J39
* JHC3 N2_231 to J37

Take your time with this step to prevent errors during the demonstration.

#### SMA or Antenna Setup
You can connect an antenna to any of the following SMA ports on the HW_XM_655 card.
* LFB_ADC_02 (10MHz to 1GHz)
* MFB_ADC_01 (1GHz to 4GHz)
* HFB_ADC_01 (4GHz to 5GHz)
* SHFB_ADC_02 (5GHz to 6GHz)

Alternatively, you can create a loopback connection using the SMA cable provided in your ZCU208 development kit. See the possible connections below:
* LMFB_DAC_01 — LFB_ADC_02 (10MHz to 1GHz)
* MFB_DAC_01 — MFB_ADC_01 (1GHz to 4GHz)
* HFB_DAC_01 — HFB_ADC_01 (4GHz to 5GHz)
* SHFB_DAC_02 — SHFB_ADC_02 (5GHz to 6GHz)

<div class="alert alert-box alert-danger">
<b>Caution:</b>
    In this demonstration, we generate tones using the RFSoC development board. Your device should be setup in loopback mode. You should understand that the RFSoC platform can also transmit RF signals wirelessly. Remember that unlicensed wireless transmission of RF signals may be illegal in your geographical location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. Note that it is also illegal to intercept and decode particular RF signals. If you are unsure, please seek professional support.
</div>

### Software Setup <a class="anchor" id="software-setup"></a>
We're nearly finished setting up the demonstration system. The majority of the libraries used by the spectrum analyzer design are contained inside the RFSoC-SAM software package. We only need to run a few code cells to initialise the software environment.

The primary module for loading the Spectrum Analyzer design is contained inside `rfsoc_sam.overlay`. The class we are interested in using is `Overlay()`. During initialisation the class downloads the Spectrum Analyzer bitstream to the PL and configures the RF DCs and FPGA IP cores contained in our system. This process may take around a minute to complete.

**Run** the code cell below to load the RFSoC-SAM Overlay class.

In [ ]:
from rfsoc_sam.overlay import Overlay

sam = Overlay()

When the RFSoC-SAM Overlay class is initialising, the setup script will also program the LMK and LMX low-jitter clock chips on the ZCU208 to 245.76MHz and 409.6MHz respectively.

Lets now initialise the analyzer, and setup user control.

In [ ]:
analyzer = sam.spectrum_analyzer()

----

## Simple Tone Generation <a class="anchor" id="simple-tone-generation"></a>
A simple amplitude controller is required to generate tones using the RF Digital-to-Analogue Converters (RF DACs). We use tone generation in this demonstration to provide a signal for the user to inspect when using the Spectrum Analyzer Module.

Run the code cell below to reveal a widget, which can be used to control the transmission frequency and amplitude.

In [ ]:
analyzer.children[2]

## The Spectrum Analyzer <a class="anchor" id="the-spectrum-analyzer"></a>
We will now explore the hardware accelerated Spectrum Analyzer Module. It is worthwhile noting the analyzers capabilities below:

* The analyzer is capable of inspecting 1966.08MHz of bandwidth.
* It can achieve a maximum spectral resolution of 0.292969kHz. 
* The bandwidth is adjustable between 1966.08MHz and 1.92MHz.
* The range of inspection is between 0 to 4915.2MHz using higher order Nyquist techniques.

In [ ]:
analyzer.children[1]

The Spectrum Analyzer Module contains a hardware accelerated FFT core, which can convert the RF sampled signal to the frequency domain using a range of different FFT lengths, $N = 64$ upto $N = 8192$. The frequency domain signal is further manipulated using a custom floating point processor to obtain the representative Power Spectral Density (PSD) or Power Spectrum. Furthermore, a hardware accelerated decibel (dB) converter is also used to condition the frequency domain signal for visual analysis.

Through the loopback connection, you should be able to use the Spectrum Analyzer Module to locate the tone you previously generated using the tone generator. If you have an antenna connected to your board, try and locate signals of interest using the Spectrum Analyzer's control widgets.

### A Simple Example <a class="anchor" id="a-simple-example"></a>
If you would like to enable stimulus for the spectrum analyzer, you can use your mobile phone to create WiFi traffic. Follow the steps below to create an interesting WiFi spectrum to visualise.
* Connect your mobile phone to an access point that uses WiFi. 
* Then configure the spectrum analyzer for a centre frequency of 2400MHz and a decimation factor of 16. 
* Switch on the spectrum analyzer and spectrogram.
* Use your phone to stream a video, or music. This will create WiFi traffic for inspection.
* Place your phone close to the RF ADC ports of the spectrum analyzer.

You should see a similar output as given in the [Figure 3](#fig-3) below.

<a class="anchor" id="fig-3"></a>
<figure>
<img src='images/wifi_example.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 3: Capturing a WiFi signal using the Spectrum Analyser Module.</b></figcaption>
</figure>

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented a hardware accelerated Spectrum Analyzer Module for the ZCU208 development board.